######
1) For Jun 2000, Planning Areas refer to areas demarcated in the Urban Redevelopment Authority's Master Plan 1998.
2) For Jun 2001 to 2010, Planning Areas refer to areas demarcated in the Urban Redevelopment Authority's Master Plan 2008.
3) For Jun 2011 to 2018, Planning Areas refer to areas demarcated in the Urban Redevelopment Authority's Master Plan 2014.
4) Data from 2003 onwards exclude residents who have been away from Singapore for a continuous period of 12 months or longer as at the reference period.
5) The figures have been rounded to the nearest 10.
6) The data may not add up due to rounding.

In [25]:
import os
import json
import pandas as pd
import geopandas as gpd

os.chdir("/Users/DarylTay/Desktop/Python/data/population-density")

In [26]:
### 2000 - 2004

old_df = pd.read_csv('singapore-residents-by-planning-area-subzone-age-group-and-sex-june-2000-onwards.csv')
old_df = old_df[['planning_area','resident_count','year']]

# Exclude 2005 as 2005 data does not have some locations
result = old_df.groupby(['planning_area','year'],as_index = False)[['resident_count']].sum()
result = result.loc[~(result['year']==2005)]
result.columns = ['planning area','year','total']
result['planning area'] = result['planning area'].str.upper()
#result

In [27]:
### 2005 - 2010
df_shape = gpd.read_file('PLAN_BDY_AGE_GENDER_2005.shp')[['PLN_AREA_N','TOTAL']]
df_shape['year'] = ['2005' for i in range(df_shape.shape[0])]

for year in range(2006,2011):
    df = gpd.read_file('PLAN_BDY_AGE_GENDER_' + str(year) + '.shp')[['PLN_AREA_N','TOTAL']]
    df['year'] = [str(year) for i in range(df.shape[0])]
    #print(str(year) + str(df.shape))
    df_shape = pd.concat([df_shape,df])

df_shape.columns = ['planning area','total','year']
total = list(df_shape['total'])
df_shape = df_shape[['planning area','year']]
df_shape['total'] = total
#df_shape

In [28]:
### 2011 - 2019 
df2 = pd.read_csv('planning-area-subzone-age-group-sex-and-type-of-dwelling-june-2011-2019.csv')
result2 = df2.groupby(['planning_area','year',],as_index = False)[['resident_count']].sum()
result2.columns = ['planning area','year','total']
result2['planning area'] = result2['planning area'].str.upper()
#result2

In [29]:
## Concatenate all together

final_df = pd.concat([result,df_shape,result2])
export = final_df.to_excel('population count 2000 - 2019.xlsx')

In [30]:
### importing of base map

map_year = ['98','08','14']

map_98 = gpd.read_file('MP98_PLNG_AREA_NO_SEA_PL.shp')
map_08 = gpd.read_file('MP08_PLNG_AREA_NO_SEA_PL.shp')
map_14 = gpd.read_file('MP14_PLNG_AREA_NO_SEA_PL.shp')

map_98.to_excel('map_98.xlsx')
map_08.to_excel('map_08.xlsx')
map_14.to_excel('map_14.xlsx')

In [31]:
from bokeh.io import output_notebook, show, curdoc
from bokeh.plotting import figure, output_file
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, Slider, HoverTool
from bokeh.palettes import brewer
from bokeh.layouts import widgetbox, row, column

#Define function that returns json_data for year selected by user.
start_year = 2019

def json_data(selectedYear):
    if selectedYear == 2000:
        final_df_yr = final_df[final_df['year'] == selectedYear]
        plot = map_98.merge(final_df_yr, left_on = 'PLN_AREA_N', right_on = 'planning area')
        merged_json = json.loads(plot.to_json())
        json_data = json.dumps(merged_json)
    elif 2001 <= selectedYear <= 2010:
        final_df_yr = final_df[final_df['year'] == selectedYear]
        plot = map_08.merge(final_df_yr, left_on = 'PLN_AREA_N', right_on = 'planning area')
        merged_json = json.loads(plot.to_json())
        json_data = json.dumps(merged_json)
    elif selectedYear >=2011:
        final_df_yr = final_df[final_df['year'] == selectedYear]
        plot = map_14.merge(final_df_yr, left_on = 'PLN_AREA_N', right_on = 'planning area')
        merged_json = json.loads(plot.to_json())
        json_data = json.dumps(merged_json)
    return json_data

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data(start_year))

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][6]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 300000)

#Define custom tick labels for color bar.
tick_labels = {'1': '< 5,000', '2': '5,000 - < 10,000', '3':'10,000 - < 50,000', '4':'50,000 - < 100,000', '5':'100,000 - < 150,000', '6':'150,000 - < 200,000', '7':'200,000 - < 250,000','8':'250,000 - < 300,000'}

#Add hover tool
hover = HoverTool(tooltips = [ ('Region','@PLN_AREA_N'),('Population Count', '@total')])

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=6,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'Population Density of Singapore, '+str(start_year), plot_height = 600 , plot_width = 950, toolbar_location = None,tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.xaxis.visible = False
p.yaxis.visible = False

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'total', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify layout
p.add_layout(color_bar, 'below')

# Define the callback function: update_plot
def update_plot(attr, old, new):
    yr = slider.value
    new_data = json_data(yr)
    geosource.geojson = new_data
    p.title.text = 'Population Density of Singapore, %d' %yr
    
# Make a slider
slider = Slider(title = 'Year',start = 2000, end = 2019, step = 1, value = start_year)
slider.on_change('value', update_plot)

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)

#Display the plot
show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



######
Note that the plot does not update when you change slider value in your Jupyter Notebook. To view this application in interactive mode you need to set up a local Bokeh server. Open a command line window in your current directory and execute bokeh serve --show filename.ipynb command.